In [2]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.8.2

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    scikit-learn-0.20.1        |   py36h22eb022_0         5.7 MB
    liblapack-3.8.0            |      11_openblas          10 KB  conda-forge
    numpy-1.18.1               |   py36h95a1406_0         5.2 MB  conda-forge
    liblapacke-3.8.0           |      11_openblas          10 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    libopenblas-0.3.6          |       h5a2b251_2         7.7 MB
    sci

In [3]:
!pip install lxml==3.4.2


     |████████████████████████████████| 3.5MB 23.3MB/s eta 0:00:01
  Stored in directory: /home/jupyterlab/.cache/pip/wheels/7a/ca/86/31e51ca690578bffcca3702c652eabd5789d34dfef18d4b065
Successfully built lxml


In [34]:
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
tables = pd.read_html(url)[0]
tables.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


In [5]:
tables.columns

Index(['Postcode', 'Borough', 'Neighbourhood'], dtype='object')

In [6]:
df=tables

In [35]:
df.drop(df[df['Borough']== "Not assigned"].index, axis=0, inplace=True)
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


In [36]:
df1=df.groupby("Postcode").agg(lambda x:','.join(set(x)))
df1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [37]:
df1.loc[df1['Neighbourhood']=="Not assigned",'Neighbourhood']=df1.loc[df1['Neighbourhood']=="Not assigned",'Borough']
df1.head()

,Borough,Neighbourhood
Postcode,,
M1B,Scarborough,"Malvern,Rouge"
M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
M1E,Scarborough,"Morningside,Guildwood,West Hill"
M1G,Scarborough,Woburn
M1H,Scarborough,Cedarbrae


In [10]:
df1.shape

(103, 2)

In [38]:
df1.reset_index(inplace = True) 
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [39]:
df1.rename(columns={'Postcode' : 'PostalCode'}, inplace=True)
df1.head()

,PostalCode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern,Rouge"
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union"
2,M1E,Scarborough,"Morningside,Guildwood,West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [13]:

df3=pd.read_csv('http://cocl.us/Geospatial_data')
df3.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [14]:
df3.dtypes



Postal Code     object
Latitude       float64
Longitude      float64
dtype: object

In [31]:
df3.rename(columns={'Postal Code' : 'PostalCode'}, inplace=True)
df3.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [54]:
df_Final=pd.merge(df1,df3, on='PostalCode')
df_Final

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern,Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek,Rouge Hill,Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Morningside,Guildwood,West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
5,M1J,Scarborough,Scarborough Village,43.744734,-79.239476
6,M1K,Scarborough,"Ionview,Kennedy Park,East Birchmount Park",43.727929,-79.262029
7,M1L,Scarborough,"Golden Mile,Clairlea,Oakridge",43.711112,-79.284577
8,M1M,Scarborough,"Cliffside,Scarborough Village West,Cliffcrest",43.716316,-79.239476
9,M1N,Scarborough,"Cliffside West,Birch Cliff",43.692657,-79.264848


### Find the Numbers of boroughs and neighborhoods.

In [18]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(df_Final['Borough'].unique()),
        df.shape[0]
    )
)

The dataframe has 10 boroughs and 210 neighborhoods.


#### Use geopy library to get the latitude and longitude values of Downtown Toronto City.

In [22]:
address = 'Downtown Toronto, ON'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Downtown Toronto City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Downtown Toronto City are 43.6563221, -79.3809161.


#### visualization of Downtown Toronto  neighborhoods 

In [33]:
# create map of New York using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighbourhood in zip(df_Final['Latitude'], df_Final['Longitude'], df_Final['Borough'], df_Final['Neighbourhood']):
    label = '{}, {}'.format(neighbourhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

#### Define Foursquare Credentials and Version

In [40]:
CLIENT_ID = '5QH431DEX2VYMOZERAM2G22WABZ5XD5GIQZHTAKY0WQ4D4OW' # your Foursquare ID
CLIENT_SECRET = 'TLZODC5EW3KXJHRL4MRCPG5Y2B4JCDSS3RFSVSF4OQJ3KWA5' # your Foursquare Secret
VERSION = '20200303' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 5QH431DEX2VYMOZERAM2G22WABZ5XD5GIQZHTAKY0WQ4D4OW
CLIENT_SECRET:TLZODC5EW3KXJHRL4MRCPG5Y2B4JCDSS3RFSVSF4OQJ3KWA5


In [41]:
toronto_data=df_Final

#### Let's explore the first neighborhood in our dataframe.

In [55]:
toronto_data.loc[45, 'Neighbourhood']

'Davisville North'

#### Get the neighborhood's latitude and longitude values.

In [57]:
neighborhood_latitude = toronto_data.loc[45, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_data.loc[45, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_data.loc[45, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of Davisville North are 43.7127511, -79.3901975.


#### Now, let's get the top 100 venues that are in Marble Hill within a radius of 500 meters.

##### First, let's create the GET request URL. Name your URL url.

In [58]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    neighborhood_latitude, 
    neighborhood_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=5QH431DEX2VYMOZERAM2G22WABZ5XD5GIQZHTAKY0WQ4D4OW&client_secret=TLZODC5EW3KXJHRL4MRCPG5Y2B4JCDSS3RFSVSF4OQJ3KWA5&v=20200303&ll=43.7127511,-79.3901975&radius=500&limit=100'

##### Send the GET request and examine the resutls

In [59]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5e5f34241835dd001b7462b4'},
 'response': {'headerLocation': 'Davisville',
  'headerFullLocation': 'Davisville, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 9,
  'suggestedBounds': {'ne': {'lat': 43.7172511045, 'lng': -79.38398344441633},
   'sw': {'lat': 43.708251095499996, 'lng': -79.39641155558367}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4e8e73c30cd6209590ae7be4',
       'name': 'Summerhill Market North',
       'location': {'address': '1054 Mount Pleasant Rd.',
        'crossStreet': 'Sheldrake Blvd',
        'lat': 43.71549914910689,
        'lng': -79.39288125988016,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.71549914910689,
          'lng': -79.39288125988016}],
  

##### From the Foursquare lab in the previous module, we know that all the information is in the items key. Before we proceed, let's borrow the get_category_type function from the Foursquare lab.

In [60]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

##### Now we are ready to clean the json and structure it into a pandas dataframe.

In [61]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,name,categories,lat,lng
0,Summerhill Market North,Food & Drink Shop,43.715499,-79.392881
1,Sherwood Park,Park,43.716551,-79.387776
2,Homeway Restaurant & Brunch,Breakfast Spot,43.712641,-79.391557
3,Winners,Department Store,43.713236,-79.393873
4,Best Western Roehampton Hotel & Suites,Hotel,43.708878,-79.390880


##### And how many venues were returned by Foursquare?

In [62]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

9 venues were returned by Foursquare.


## 2. Explore Neighborhoods in Toronto